In [ ]:
!pip install lightning torchvision einops

In [1]:
try:
    from key_utils import KeyClfDataModule, KeyClf
    from models.resnet import resnet101
except:
    import sys
    sys.path.append("/kaggle/input/keystroke-util")
    from key_utils import KeyClfDataModule, KeyClf
    from models.resnet import resnet101

from lightning.pytorch.callbacks import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger
import torchvision
import torchvision.transforms.functional
import lightning as L

def transforms(frames):
    h, w = frames.shape[-2], frames.shape[-1]

    frames = torchvision.transforms.functional.resized_crop(
                              frames, 
                              top=h//2, 
                              left=0, 
                              height=h//2,
                              width=w,
                              size=(224, 224),
                              antialias=True)
    
    frames = frames.permute(1, 0, 2, 3)
    frames = frames / 255.0
    return frames.float()

dm = KeyClfDataModule(segment_dir='datasets/angle/segments_dir', 
                      num_workers=0,
                      transforms=transforms)

weights = dm.train_weights

model_name = "resnet_101_clf"

class ResnetKeyClf(KeyClf):
    def __init__(self, model_name, id2label, label2id, learning_rate=0.01):
        super().__init__(weights, model_name=model_name, id2label=id2label, label2id=label2id, learning_rate=learning_rate)
        
        self.model = resnet101(sample_size=224, sample_duration=8, num_classes=len(id2label))

trainer = L.Trainer(
    accelerator='cpu',
    devices='auto',
    fast_dev_run=True,
    log_every_n_steps=100,
    callbacks=EarlyStopping(monitor='val_loss', patience=5),
    max_epochs=100,
)

model = ResnetKeyClf(model_name, dm.id2label, dm.label2id)
trainer.fit(model, dm)

/Users/haily/.pyenv/versions/3.10.4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/haily/.pyenv/versions/3.10.4/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


Train:
 Counter({3: 1243, 8: 635, 2: 591, 12: 442, 4: 415, 23: 395, 18: 364, 21: 330, 17: 328, 22: 263, 24: 244, 15: 236, 11: 231, 6: 210, 7: 202, 16: 159, 28: 158, 10: 149, 26: 134, 9: 133, 19: 121, 5: 120, 1: 104, 0: 94, 14: 93, 25: 89, 20: 75, 29: 75, 27: 75, 13: 66})
Val:
 Counter({3: 143, 8: 90, 2: 66, 18: 59, 12: 58, 23: 50, 4: 46, 21: 39, 22: 38, 17: 36, 24: 29, 15: 28, 19: 26, 6: 24, 16: 22, 11: 22, 7: 20, 10: 19, 26: 19, 28: 18, 9: 17, 25: 16, 5: 15, 14: 14, 1: 12, 13: 12, 29: 12, 0: 10, 27: 7, 20: 5})
Test:
 Counter({3: 158, 8: 81, 2: 72, 18: 60, 12: 52, 4: 47, 23: 43, 17: 42, 22: 40, 21: 38, 15: 33, 7: 32, 6: 27, 24: 22, 16: 20, 28: 20, 1: 19, 14: 18, 11: 18, 19: 18, 26: 15, 9: 14, 5: 13, 25: 13, 10: 12, 13: 11, 27: 10, 29: 8, 0: 8, 20: 8})
train_weights: 
 [2.756737588652482, 2.4916666666666667, 0.43846587704455725, 0.20847412174845803, 0.6244176706827309, 2.1594444444444445, 1.233968253968254, 1.282838283828383, 0.40808398950131236, 1.9483709273182956, 1.7391498881431768, 

/Users/haily/Documents/GitHub/Research Learning/models/resnet.py:130: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | loss_fn   | CrossEntropyLoss   | 0      | train
1 | train_acc | MulticlassAccuracy | 0      | train
2 | val_acc   | MulticlassAccuracy | 0      | train
3 | test_acc  | MulticlassAccuracy | 0      | train
4 | model     | ResNet             | 85.3 M | train
---------------------------------------------------------
85.3 M    Trainable params
0         Non-trainable params
85.3 M    Total params
341.226   Total estimated model params size (MB)
/Users/haily/.pyenv/versions/3.10.4/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increas

Epoch 0: 100%|██████████| 1/1 [00:04<00:00,  0.21it/s, val_loss=4.32e+12, val_acc=0.000, train_loss=4.040, train_acc=0.000]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0, train_acc: 0.0
Epoch 0: 100%|██████████| 1/1 [00:04<00:00,  0.21it/s, val_loss=4.32e+12, val_acc=0.000, train_loss=4.040, train_acc=0.000]


In [2]:
trainer.test(model, dm)

/Users/haily/.pyenv/versions/3.10.4/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.01it/s]
              precision    recall  f1-score   support

           d       0.00      0.00      0.00       0.0
           l       0.00      0.00      0.00       1.0
           t       0.00      0.00      0.00       1.0
           x       0.00      0.00      0.00       1.0
           z       0.00      0.00      0.00       1.0

    accuracy                           0.00       4.0
   macro avg       0.00      0.00      0.00       4.0
weighted avg       0.00      0.00      0.00       4.0



/Users/haily/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/haily/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/haily/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} 

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │      4760178524160.0      │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 4760178524160.0}]

In [2]:
import subprocess
subprocess.run(["zip", "-r", f"{model_name}.zip", "./lightning_logs"])

  adding: lightning_logs/

CompletedProcess(args=['zip', '-r', 'resnet_101_clf.zip', './lightning_logs'], returncode=0)

 (stored 0%)
  adding: lightning_logs/version_0/ (stored 0%)
  adding: lightning_logs/version_0/events.out.tfevents.1723256238.Lys-Air-3.cogeco.local.97033.0 (deflated 34%)
  adding: lightning_logs/version_0/hparams.yaml (deflated 51%)
